In [2]:
from readDB import *
from preProcessing import preProcessing
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import operator


def fn_nvarFilt(poses, nvar):
    nvjr = list(nvar)
    ja = lambda tag: tag if tag != 'j' else 'a'
    nvars = [(p0, ja(p1[:1].lower())) for p0, p1 in poses if p1[:1] in nvjr]
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(nvar[0], pos=nvar[1]) for nvar in nvars]

def FindN(query):
    tokens = word_tokenize(query.lower())
    poses = pos_tag(tokens)
    nltk_res = fn_nvarFilt(poses, 'N')
    nltk_freq = defaultdict(int)
    for token in nltk_res:
        nltk_freq[token] += 1
    return nltk_res

# FindN 을 사용하여 쿼리에서 명사를 추출하여 Boolean에 사용

q=input() #쿼리 입력 ex) warm chicken with onion

candidate=FindN(q)   # candidate = [chicken,onion]
print(candidate)


chicken with onion and salt
['chicken', 'onion', 'salt']


In [3]:

### 이 경우는 무조건 쿼리에 적힌 재료들이 포함되어 있는 경우, 
###예를들어 양파 닭 소금이 쿼리에 있다면 저 3개를 모두 가지고 있는 결과물 중에서 검색

def boolean_Ingredient_all(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu not in s:
                break;
            else:
                flag += 1 
        if flag == len(q_ing):
            querylen =len(q_ing)
            inglen=len(s.split(' '))
            rank_value = querylen/inglen
            lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    return lg

def boolean_Ingredient_nameonly(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu not in s:
                break;
            else:
                flag += 1 
        if flag == len(q_ing):
            querylen =len(q_ing)
            inglen=len(s.split(' '))
            rank_value = querylen/inglen
            lg.append(food)
    return lg

def boolean_Ingredient(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu not in s:
                break;
            else:
                flag += 1 
        if flag == len(q_ing):
            querylen =len(q_ing)
            inglen=len(s.split(' '))
            rank_value = querylen/inglen
            lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    for i in range(5):
        if (len(lg)-i-1) == -1:
            break;
        
        ranked.append(lg[len(lg)-1-i][0])
        
    return ranked

def boolean_Ingredient_weight(q_ing, db_ing) :
    ranked_with_weight = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu not in s:
                break;
            else:
                flag += 1 
        if flag == len(q_ing):
            querylen =len(q_ing)
            inglen=len(s.split(' '))
            rank_value = querylen/inglen
            lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    for i in range(5):
        if (len(lg)-i-1) == -1:
            break;
        ranked_with_weight.append(lg[len(lg)-i-1])
        
    return ranked_with_weight
data_name = boolean_Ingredient_nameonly(candidate, db) # 소팅되지 않은 해당 음식들 전부
data_all = boolean_Ingredient_all(candidate, db) # 소팅되지 않은 음식들의 값 전부 계산
data = boolean_Ingredient(candidate, db) # 소팅된 음식들의 weight 빼고
data_weight = boolean_Ingredient_weight(candidate, db) # data에는 chicken과 onion이 포함된 커리만 출력됨 - ranking을 통해 5위까지 출력
print(data_name)
print(data_all)
print(data)
print(data_weight)

['Coconut Curry Chicken Soup', 'Slow Cooker Indian Butter Chicken', 'Easy Chicken Tikka Masala', 'Instant Pot Mushroom Risotto', 'Italian Wedding Soup', 'Instant Pot Chicken Tortilla Soup', 'Roasted Shrimp Enchiladas with Jalapeno Cream Sauce', 'Thai Red Curry Noodle Soup', 'Thai Green Curry Chicken Soup', 'Thai Sweet Potato and Carrot Soup', 'Slow Cooker Corn Chowder']
[['Italian Wedding Soup', 0.03225806451612903], ['Roasted Shrimp Enchiladas with Jalapeno Cream Sauce', 0.03571428571428571], ['Thai Red Curry Noodle Soup', 0.041666666666666664], ['Instant Pot Chicken Tortilla Soup', 0.04477611940298507], ['Easy Chicken Tikka Masala', 0.05172413793103448], ['Slow Cooker Indian Butter Chicken', 0.05263157894736842], ['Thai Sweet Potato and Carrot Soup', 0.05357142857142857], ['Thai Green Curry Chicken Soup', 0.05454545454545454], ['Coconut Curry Chicken Soup', 0.057692307692307696], ['Instant Pot Mushroom Risotto', 0.07317073170731707], ['Slow Cooker Corn Chowder', 0.075]]
['Slow Cooker

In [13]:

### 이 경우는 쿼리에 적힌 재료들이 모두 포함되어 있지 않더라도 가까운 경우, 
###예를들어 양파 닭 소금이 쿼리에 있다면 저 2개만을 가지고 있더라도 가까운 음식 중에서 검색

def NA_Ingredient_all(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu in s:
                flag +=1

        flaglen = flag
        inglen=len(s.split(' '))
        rank_value = flaglen/inglen
        lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    return lg

def NA_Ingredient_nameonly(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu in s:
                flag +=1
        flaglen =flag
        inglen=len(s.split(' '))
        rank_value = flaglen/inglen
        lg.append(food)
    return lg

def NA_Ingredient(q_ing, db_ing) :
    ranked = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu in s:
                flag +=1
        flaglen = flag
        inglen=len(s.split(' '))
        rank_value = flaglen/inglen
        lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    
    for i in range(5):
        if (len(lg)-i-1) == -1:
            break;
        ranked.append(lg[len(lg)-1-i][0])
        
    return ranked

def NA_Ingredient_weight(q_ing, db_ing) :
    ranked_with_weight = []
    lg =[]
    for food in db.keys():
        flag = 0
        s = db[food]['ingredient']
        for qu in (q_ing):
            if qu in s:
                flag+=1;
                
        flaglen = flag
        inglen=len(s.split(' '))
        rank_value = flaglen/inglen
        lg.append([food, rank_value])
    
    lg =sorted(lg, key=operator.itemgetter(1))
    for i in range(5):
        if (len(lg)-i-1) == -1:
            break;
        ranked_with_weight.append(lg[len(lg)-i-1])
        
    return ranked_with_weight
NAdata_name = NA_Ingredient_nameonly(candidate, db) # 소팅되지 않은 해당 음식들 전부
NAdata_all = NA_Ingredient_all(candidate, db) # 소팅되지 않은 음식들의 값 전부 계산
NAdata = NA_Ingredient(candidate, db) # 소팅된 음식들의 weight 빼고
NAdata_weight = NA_Ingredient_weight(candidate, db) # data에는 chicken과 onion이 포함된 커리만 출력됨 - ranking을 통해 5위까지 출력
print(NAdata_name)
print(NAdata_all)
print(NAdata)
print(NAdata_weight)

['Deep Dish Chicago Style Pizza', 'Bacon and Egg Toast Cups', 'Ham Egg and Cheese Pockets', 'White Mushroom Pizza', 'Reuben Grilled Cheese', 'Best Ever Classic Macaroni Salad', 'Baby Back Ribs with Balsamic Peach BBQ Sauce', 'Stuffed Green Chili Con Queso Cheeseburger Sliders', 'Bacon Breakfast Pizza', 'Sema Chicken', 'Miso Soup With Vermicelli, Mushrooms And Tofu', 'Shrimp Potstickers', 'Vegetable Potstickers', 'Pork Potstickers', 'Garlic Ginger Chicken Potstickers', 'Sesame Chicken Potstickers', 'Potstickers', 'Shrimp Dumplings', 'Bacon Wrapped Teriyaki Hot Dogs With Pineapple Salsa', 'Asian-Style Cobb Salad', 'Hoisin Asian Meatballs', 'Roasted Shrimp Quinoa Spring Rolls', 'Coconut Curry Chicken Soup', 'Quinoa Veggie Fride Rice', 'Pork And Asparagus With Hoisin Sauce', 'Asian Style Chicken Nuggets With Lemon Glaze', 'Honey Teriyaki Hot Wings', 'Pork B1elly Dliders', 'Sweet And Sour Stir-Fried Chicken', 'Pinoy Spaghetti', 'Pancetta Fried Rice', 'Asian Quinoa salad', 'Tofu soba noodles